In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
%matplotlib inline

In [2]:
import geopandas as gpd
from shapely.geometry import Point
from bokeh.plotting import figure, output_notebook, show
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from ipywidgets import interact

In [3]:
# Notebook Styling 
sns.set()
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
covid_deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
covid_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
covid_recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [5]:
def preprocess_covid_data(df=covid_deaths_df):
    df=df.copy()
    df['location'] = df.apply(lambda x: f"{x['Province/State']}, {x['Country/Region']}" if pd.notnull(x['Province/State']) else x['Country/Region'], axis=1)
    locations_df = df[['location', 'Lat', 'Long']].copy()
    df.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], axis=1, inplace=True)
    df.set_index('location', inplace=True)
    df.index.name = None
    df = df.T
    df.reset_index(inplace=True)
    df.rename({'index':'date'}, axis=1, inplace=True)
    df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')
    totals_df = df[[col for col in df.columns if col != 'date']].cumsum().copy()
    totals_df.set_index(df['date'], inplace=True)
    totals_df.reset_index(inplace=True)
    return (locations_df, totals_df)                            

In [6]:
death_locations, deaths_df = preprocess_covid_data(df=covid_deaths_df)
confirmed_locations, confirmed_df = preprocess_covid_data(df=covid_confirmed_df)
recovered_locations, recovered_df = preprocess_covid_data(df=covid_recovered_df)

In [7]:
deaths_df.tail(3)

,date,Thailand,Japan,Singapore,Nepal,Malaysia,"British Columbia, Canada","New South Wales, Australia","Victoria, Australia","Queensland, Australia",Cambodia,Sri Lanka,Germany,Finland,United Arab Emirates,Philippines,India,Italy,Sweden,Spain,"South Australia, Australia",Belgium,Egypt,"From Diamond Princess, Australia",Lebanon,Iraq,Oman,Afghanistan,Bahrain,Kuwait,Algeria,Croatia,Switzerland,Austria,Israel,Pakistan,Brazil,Georgia,Greece,North Macedonia,Norway,Romania,Estonia,San Marino,Belarus,Iceland,Lithuania,Mexico,New Zealand,Nigeria,"Western Australia, Australia",Ireland,Luxembourg,Monaco,Qatar,Ecuador,Azerbaijan,Armenia,Dominican Republic,Indonesia,Portugal,Andorra,"Tasmania, Australia",Latvia,Morocco,Saudi Arabia,Senegal,Argentina,Chile,Jordan,Ukraine,Hungary,"Northern Territory, Australia",Liechtenstein,Poland,Tunisia,Bosnia and Herzegovina,Slovenia,South Africa,Bhutan,Cameroon,Colombia,Costa Rica,Peru,Serbia,Slovakia,Togo,Malta,Martinique,Bulgaria,Maldives,Bangladesh,Paraguay,"Ontario, Canada","Alberta, Canada","Quebec, Canada",Albania,Cyprus,Brunei,"Washington, US","New York, US","California, US","Massachusetts, US","Diamond Princess, US","Grand Princess, US","Georgia, US","Colorado, US","Florida, US","New Jersey, US","Oregon, US","Texas, US","Illinois, US","Pennsylvania, US","Iowa, US","Maryland, US","North Carolina, US","South Carolina, US","Tennessee, US","Virginia, US","Arizona, US","Indiana, US","Kentucky, US","District of Columbia, US","Nevada, US","New Hampshire, US","Minnesota, US","Nebraska, US","Ohio, US","Rhode Island, US","Wisconsin, US","Connecticut, US","Hawaii, US","Oklahoma, US","Utah, US",Burkina Faso,Holy See,Mongolia,Panama,"Kansas, US","Louisiana, US","Missouri, US","Vermont, US","Alaska, US","Arkansas, US","Delaware, US","Idaho, US","Maine, US","Michigan, US","Mississippi, US","Montana, US","New Mexico, US","North Dakota, US","South Dakota, US","West Virginia, US","Wyoming, US","Hubei, China",Iran,"Korea, South","France, France","Guangdong, China","Henan, China","Zhejiang, China","Hunan, China","Anhui, China","Jiangxi, China","Shandong, China","Diamond Princess, Cruise Ship","Jiangsu, China","Chongqing, China","Sichuan, China","Heilongjiang, China","Denmark, Denmark","Beijing, China","Shanghai, China","Hebei, China","Fujian, China","Guangxi, China","Shaanxi, China","Yunnan, China","Hainan, China","Guizhou, China","Tianjin, China","Shanxi, China","Gansu, China","Hong Kong, China","Liaoning, China","Jilin, China",Czechia,"Xinjiang, China","Inner Mongolia, China","Ningxia, China",Taiwan*,Vietnam,Russia,"Qinghai, China","Macau, China",Moldova,Bolivia,"Faroe Islands, Denmark","St Martin, France",Honduras,"Channel Islands, United Kingdom","New Brunswick, Canada","Tibet, China",Congo (Kinshasa),Cote d'Ivoire,"Saint Barthelemy, France",Jamaica,Reunion,Turkey,"Gibraltar, United Kingdom","Kitsap, WA, US","Solano, CA, US","Santa Cruz, CA, US","Napa, CA, US","Ventura, CA, US","Worcester, MA, US","Gwinnett, GA, US","DeKalb, GA, US","Floyd, GA, US","Fayette, GA, US","Gregg, TX, US","Monmouth, NJ, US","Burlington, NJ, US","Camden, NJ, US","Passaic, NJ, US","Union, NJ, US","Eagle, CO, US","Larimer, CO, US","Arapahoe, CO, US","Gunnison, CO, US","Kane, IL, US","Monroe, PA, US","Philadelphia, PA, US","Norfolk, VA, US","Arlington, VA, US","Spotsylvania, VA, US","Loudoun, VA, US","Prince George's, MD, US","Pottawattamie, IA, US","Camden, NC, US","Pima, AZ, US","Noble, IN, US","Adams, IN, US","Boone, IN, US","Dane, WI, US","Pierce, WI, US","Cuyahoga, OH, US","Weber, UT, US","Bennington County, VT, US","Carver County, MN, US","Charlotte County, FL, US","Cherokee County, GA, US","Collin County, TX, US","Jefferson County, KY, US","Jefferson Parish, LA, US","Shasta County, CA, US","Spartanburg County, SC, US","Harrison County, KY, US","Johnson County, IA, US","Berkshire County, MA, US","Davidson County, TN, US","Douglas County, OR, US","Fresno County, CA, US","Harford County, MD, US","Hendricks County, IN, US","Hudson County

In [8]:
confirmed_df.tail(3)

,date,Thailand,Japan,Singapore,Nepal,Malaysia,"British Columbia, Canada","New South Wales, Australia","Victoria, Australia","Queensland, Australia",Cambodia,Sri Lanka,Germany,Finland,United Arab Emirates,Philippines,India,Italy,Sweden,Spain,"South Australia, Australia",Belgium,Egypt,"From Diamond Princess, Australia",Lebanon,Iraq,Oman,Afghanistan,Bahrain,Kuwait,Algeria,Croatia,Switzerland,Austria,Israel,Pakistan,Brazil,Georgia,Greece,North Macedonia,Norway,Romania,Estonia,San Marino,Belarus,Iceland,Lithuania,Mexico,New Zealand,Nigeria,"Western Australia, Australia",Ireland,Luxembourg,Monaco,Qatar,Ecuador,Azerbaijan,Armenia,Dominican Republic,Indonesia,Portugal,Andorra,"Tasmania, Australia",Latvia,Morocco,Saudi Arabia,Senegal,Argentina,Chile,Jordan,Ukraine,Hungary,"Northern Territory, Australia",Liechtenstein,Poland,Tunisia,Bosnia and Herzegovina,Slovenia,South Africa,Bhutan,Cameroon,Colombia,Costa Rica,Peru,Serbia,Slovakia,Togo,Malta,Martinique,Bulgaria,Maldives,Bangladesh,Paraguay,"Ontario, Canada","Alberta, Canada","Quebec, Canada",Albania,Cyprus,Brunei,"Washington, US","New York, US","California, US","Massachusetts, US","Diamond Princess, US","Grand Princess, US","Georgia, US","Colorado, US","Florida, US","New Jersey, US","Oregon, US","Texas, US","Illinois, US","Pennsylvania, US","Iowa, US","Maryland, US","North Carolina, US","South Carolina, US","Tennessee, US","Virginia, US","Arizona, US","Indiana, US","Kentucky, US","District of Columbia, US","Nevada, US","New Hampshire, US","Minnesota, US","Nebraska, US","Ohio, US","Rhode Island, US","Wisconsin, US","Connecticut, US","Hawaii, US","Oklahoma, US","Utah, US",Burkina Faso,Holy See,Mongolia,Panama,"Kansas, US","Louisiana, US","Missouri, US","Vermont, US","Alaska, US","Arkansas, US","Delaware, US","Idaho, US","Maine, US","Michigan, US","Mississippi, US","Montana, US","New Mexico, US","North Dakota, US","South Dakota, US","West Virginia, US","Wyoming, US","Hubei, China",Iran,"Korea, South","France, France","Guangdong, China","Henan, China","Zhejiang, China","Hunan, China","Anhui, China","Jiangxi, China","Shandong, China","Diamond Princess, Cruise Ship","Jiangsu, China","Chongqing, China","Sichuan, China","Heilongjiang, China","Denmark, Denmark","Beijing, China","Shanghai, China","Hebei, China","Fujian, China","Guangxi, China","Shaanxi, China","Yunnan, China","Hainan, China","Guizhou, China","Tianjin, China","Shanxi, China","Gansu, China","Hong Kong, China","Liaoning, China","Jilin, China",Czechia,"Xinjiang, China","Inner Mongolia, China","Ningxia, China",Taiwan*,Vietnam,Russia,"Qinghai, China","Macau, China",Moldova,Bolivia,"Faroe Islands, Denmark","St Martin, France",Honduras,"Channel Islands, United Kingdom","New Brunswick, Canada","Tibet, China",Congo (Kinshasa),Cote d'Ivoire,"Saint Barthelemy, France",Jamaica,Reunion,Turkey,"Gibraltar, United Kingdom","Kitsap, WA, US","Solano, CA, US","Santa Cruz, CA, US","Napa, CA, US","Ventura, CA, US","Worcester, MA, US","Gwinnett, GA, US","DeKalb, GA, US","Floyd, GA, US","Fayette, GA, US","Gregg, TX, US","Monmouth, NJ, US","Burlington, NJ, US","Camden, NJ, US","Passaic, NJ, US","Union, NJ, US","Eagle, CO, US","Larimer, CO, US","Arapahoe, CO, US","Gunnison, CO, US","Kane, IL, US","Monroe, PA, US","Philadelphia, PA, US","Norfolk, VA, US","Arlington, VA, US","Spotsylvania, VA, US","Loudoun, VA, US","Prince George's, MD, US","Pottawattamie, IA, US","Camden, NC, US","Pima, AZ, US","Noble, IN, US","Adams, IN, US","Boone, IN, US","Dane, WI, US","Pierce, WI, US","Cuyahoga, OH, US","Weber, UT, US","Bennington County, VT, US","Carver County, MN, US","Charlotte County, FL, US","Cherokee County, GA, US","Collin County, TX, US","Jefferson County, KY, US","Jefferson Parish, LA, US","Shasta County, CA, US","Spartanburg County, SC, US","Harrison County, KY, US","Johnson County, IA, US","Berkshire County, MA, US","Davidson County, TN, US","Douglas County, OR, US","Fresno County, CA, US","Harford County, MD, US","Hendricks County, IN, US","Hudson County

In [9]:
recovered_df.tail(3)

,date,Thailand,Japan,Singapore,Nepal,Malaysia,"British Columbia, Canada","New South Wales, Australia","Victoria, Australia","Queensland, Australia",Cambodia,Sri Lanka,Germany,Finland,United Arab Emirates,Philippines,India,Italy,Sweden,Spain,"South Australia, Australia",Belgium,Egypt,"From Diamond Princess, Australia",Lebanon,Iraq,Oman,Afghanistan,Bahrain,Kuwait,Algeria,Croatia,Switzerland,Austria,Israel,Pakistan,Brazil,Georgia,Greece,North Macedonia,Norway,Romania,Estonia,San Marino,Belarus,Iceland,Lithuania,Mexico,New Zealand,Nigeria,"Western Australia, Australia",Ireland,Luxembourg,Monaco,Qatar,Ecuador,Azerbaijan,Armenia,Dominican Republic,Indonesia,Portugal,Andorra,"Tasmania, Australia",Latvia,Morocco,Saudi Arabia,Senegal,Argentina,Chile,Jordan,Ukraine,Hungary,"Northern Territory, Australia",Liechtenstein,Poland,Tunisia,Bosnia and Herzegovina,Slovenia,South Africa,Bhutan,Cameroon,Colombia,Costa Rica,Peru,Serbia,Slovakia,Togo,Malta,Martinique,Bulgaria,Maldives,Bangladesh,Paraguay,"Ontario, Canada","Alberta, Canada","Quebec, Canada",Albania,Cyprus,Brunei,"Washington, US","New York, US","California, US","Massachusetts, US","Diamond Princess, US","Grand Princess, US","Georgia, US","Colorado, US","Florida, US","New Jersey, US","Oregon, US","Texas, US","Illinois, US","Pennsylvania, US","Iowa, US","Maryland, US","North Carolina, US","South Carolina, US","Tennessee, US","Virginia, US","Arizona, US","Indiana, US","Kentucky, US","District of Columbia, US","Nevada, US","New Hampshire, US","Minnesota, US","Nebraska, US","Ohio, US","Rhode Island, US","Wisconsin, US","Connecticut, US","Hawaii, US","Oklahoma, US","Utah, US",Burkina Faso,Holy See,Mongolia,Panama,"Kansas, US","Louisiana, US","Missouri, US","Vermont, US","Alaska, US","Arkansas, US","Delaware, US","Idaho, US","Maine, US","Michigan, US","Mississippi, US","Montana, US","New Mexico, US","North Dakota, US","South Dakota, US","West Virginia, US","Wyoming, US","Hubei, China",Iran,"Korea, South","France, France","Guangdong, China","Henan, China","Zhejiang, China","Hunan, China","Anhui, China","Jiangxi, China","Shandong, China","Diamond Princess, Cruise Ship","Jiangsu, China","Chongqing, China","Sichuan, China","Heilongjiang, China","Denmark, Denmark","Beijing, China","Shanghai, China","Hebei, China","Fujian, China","Guangxi, China","Shaanxi, China","Yunnan, China","Hainan, China","Guizhou, China","Tianjin, China","Shanxi, China","Gansu, China","Hong Kong, China","Liaoning, China","Jilin, China",Czechia,"Xinjiang, China","Inner Mongolia, China","Ningxia, China",Taiwan*,Vietnam,Russia,"Qinghai, China","Macau, China",Moldova,Bolivia,"Faroe Islands, Denmark","St Martin, France",Honduras,"Channel Islands, United Kingdom","New Brunswick, Canada","Tibet, China",Congo (Kinshasa),Cote d'Ivoire,"Saint Barthelemy, France",Jamaica,Reunion,Turkey,"Gibraltar, United Kingdom","Kitsap, WA, US","Solano, CA, US","Santa Cruz, CA, US","Napa, CA, US","Ventura, CA, US","Worcester, MA, US","Gwinnett, GA, US","DeKalb, GA, US","Floyd, GA, US","Fayette, GA, US","Gregg, TX, US","Monmouth, NJ, US","Burlington, NJ, US","Camden, NJ, US","Passaic, NJ, US","Union, NJ, US","Eagle, CO, US","Larimer, CO, US","Arapahoe, CO, US","Gunnison, CO, US","Kane, IL, US","Monroe, PA, US","Philadelphia, PA, US","Norfolk, VA, US","Arlington, VA, US","Spotsylvania, VA, US","Loudoun, VA, US","Prince George's, MD, US","Pottawattamie, IA, US","Camden, NC, US","Pima, AZ, US","Noble, IN, US","Adams, IN, US","Boone, IN, US","Dane, WI, US","Pierce, WI, US","Cuyahoga, OH, US","Weber, UT, US","Bennington County, VT, US","Carver County, MN, US","Charlotte County, FL, US","Cherokee County, GA, US","Collin County, TX, US","Jefferson County, KY, US","Jefferson Parish, LA, US","Shasta County, CA, US","Spartanburg County, SC, US","Harrison County, KY, US","Johnson County, IA, US","Berkshire County, MA, US","Davidson County, TN, US","Douglas County, OR, US","Fresno County, CA, US","Harford County, MD, US","Hendricks County, IN, US","Hudson County

In [10]:
top_deaths = deaths_df.loc[:, deaths_df.columns != 'date'].tail(1).T
top_deaths.rename({top_deaths.columns[0]:'total'}, axis=1, inplace=True)
top_deaths.sort_values(by='total', ascending=False, inplace=True)
top_deaths.head(20)

,total
"Hubei, China",96076.0
Italy,13243.0
Iran,6031.0
Spain,1702.0
"Korea, South",964.0
"France, France",782.0
"Henan, China",724.0
"Heilongjiang, China",480.0
"Washington, US",300.0
Japan,246.0


In [13]:
def plot_covid_19_data(location=None, case_type='Confirmed', y_type='log'):
    if case_type.lower() == 'fatal':
        df = deaths_df
    elif case_type.lower() == 'recovered':
        df = recovered_df
    else:
        df = confirmed_df
    
    nonzero_df = [False]
    nonzero_df.extend((df.sum(axis=0) > 0).tolist())
    nonzero_cols = ['date']
    nonzero_cols.extend(df.columns[nonzero_df])
    df = df[nonzero_cols].copy()

    
    if (location is None) and ('Cook County, IL, US' in list(df.columns)):
        location='Cook County, IL, US'
    else:
        location='Italy'
    
    cols = list(df.columns)
    cols.sort()
    locations = [col for col in cols if col != 'date']
    source = ColumnDataSource(df)
    date = source.data['date']
    location_data = source.data[location]
    plot = figure(x_axis_type='datetime', x_axis_label='Date', y_axis_label=f"{case_type} COVID-19 Cases", 
                  y_axis_type=y_type, title=f"{case_type} Cases of COVID-19 in {location}", plot_width=800, plot_height=500)
    line = plot.line(date, location_data)
        
    plot.title.text_font_size='18pt'
    plot.xaxis.axis_label_text_font_size = "16pt"
    plot.yaxis.axis_label_text_font_size = "16pt"
    
    plot.xaxis.axis_label_text_font_style = 'normal'
    plot.yaxis.axis_label_text_font_style = 'normal'
    
    plot.ygrid.minor_grid_line_color = 'black'
    plot.ygrid.minor_grid_line_alpha = 0.1
    
    output_notebook() 
    _ = show(plot, notebook_handle=True) 
    
    def update_covid_19_plot(location=location):
        plot.title.text = f"{case_type} Cases of COVID-19 in {location}"
        line.data_source.data['y'] = source.data[location]
        push_notebook()
    
    interact(update_covid_19_plot, location=locations)
       

In [14]:
plot_covid_19_data(case_type='Confirmed')

Loading BokehJS ...

interactive(children=(Dropdown(description='location', index=60, options=('Afghanistan', 'Alabama, US', 'Alame…

In [15]:
plot_covid_19_data(case_type='Confirmed', y_type='linear')

Loading BokehJS ...

interactive(children=(Dropdown(description='location', index=60, options=('Afghanistan', 'Alabama, US', 'Alame…

In [16]:
plot_covid_19_data(case_type='Fatal')

Loading BokehJS ...

interactive(children=(Dropdown(description='location', index=51, options=('Albania', 'Algeria', 'Anhui, China'…

In [17]:
plot_covid_19_data(case_type='Recovered')

Loading BokehJS ...

interactive(children=(Dropdown(description='location', index=21, options=('Afghanistan', 'Algeria', 'Andorra',…